In [76]:
from bse_dsdm.preprocessing.feature_scaling import *
from bse_dsdm.preprocessing.outlier_detection import *
from bse_dsdm.preprocessing.value_encoder import *
from bse_dsdm.preprocessing.value_imputer import *
from bse_dsdm.preprocessing.load_data import *
from bse_dsdm.preprocessing.exploratory_analysis import *
import pandas as pd
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score



LOAD DATA

In [77]:
df=load_csv_data('../train.csv')
print(df)

          id   short_name  overall  potential   value_eur  wage_eur  \
0     216302    E. García       71         71   1400000.0     10000   
1     237867   D. Cancola       65         71   1000000.0      2000   
2     253472      E. Kahl       65         77   1600000.0      2000   
3     223994    S. Mugoša       72         72   2300000.0      5000   
4     251635  A. Țigănașu       65         65    525000.0      3000   
...      ...          ...      ...        ...         ...       ...   
6890  242007    C. Whelan       59         66    425000.0      2000   
6891  187961     Paulinho       83         83  28500000.0     61000   
6892  257234   Y. Hamache       70         80   3400000.0      6000   
6893  232511    S. Sasaki       71         71   1300000.0      7000   
6894  246750    S. Haudum       66         70   1100000.0      4000   

     birthday_date  height_cm  weight_kg                  club_name  ...  \
0       1989-12-28        176         73  Club Atlético de San Luis  ..

In [78]:
df=remove_columns_with_na(df,80)
print(df)


          id   short_name  overall  potential   value_eur  wage_eur  \
0     216302    E. García       71         71   1400000.0     10000   
1     237867   D. Cancola       65         71   1000000.0      2000   
2     253472      E. Kahl       65         77   1600000.0      2000   
3     223994    S. Mugoša       72         72   2300000.0      5000   
4     251635  A. Țigănașu       65         65    525000.0      3000   
...      ...          ...      ...        ...         ...       ...   
6890  242007    C. Whelan       59         66    425000.0      2000   
6891  187961     Paulinho       83         83  28500000.0     61000   
6892  257234   Y. Hamache       70         80   3400000.0      6000   
6893  232511    S. Sasaki       71         71   1300000.0      7000   
6894  246750    S. Haudum       66         70   1100000.0      4000   

     birthday_date  height_cm  weight_kg                  club_name  ...  \
0       1989-12-28        176         73  Club Atlético de San Luis  ..

Preprocessing

In [79]:
categorical_columns_na=columns_with_na(df)[0]
print(categorical_columns_na)
numerical_columns_na=columns_with_na(df)[1]
print(numerical_columns_na)


['club_joined', 'player_traits']
['value_eur', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']


In [80]:
df=impute_values(df,numerical_columns_na, impute_type='mean')

c:\users\usuario\onedrive\escritorio\bse\computer.science\new project computer science\bse_dsdm\bse_dsdm\preprocessing\value_imputer.py:26: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  filled_df = df.fillna(df.mean())


In [81]:
df=impute_categorical(df,categorical_columns_na)

In [82]:
mapping={'work_rate': {'Low/Low': 1, 'Low/Medium': 2, 'Low/High': 3, 'Medium/Low': 4, 'Medium/Medium': 5, 'Medium/High': 6, 'High/Low': 7, 'High/Medium': 8, 'High/High': 9}}

df = encode_ordinal_columns(df, columns=['work_rate'], mapping=mapping)

In [83]:
df['nationality_name'] = target_encode_column(df, 'nationality_name', 'height_cm', compute_type='mean')

In [84]:
df=encode_categorical_columns(df,column='preferred_foot')

c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [85]:
df = convert_to_age(df, 'birthday_date',2021)

In [86]:
df=extract_string(df,'body_type')

In [87]:
feature_columns=['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle']
target_column=['position']
X=df[feature_columns]
y=df['position']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


In [88]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

class RandomForestModel:
    def __init__(self, n_estimators=100, random_state=42):
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)

    def fit(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        return self.model.predict(X_test)

    def evaluate(self, X_test, y_test):
        predictions = self.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        return mse


rf_model = RandomForestModel()

# Fit the model
rf_model.fit(X_train, y_train)

# Evaluate the model
mse = rf_model.evaluate(X_test, y_test)
print(f"Mean Squared Error (MSE) on test set: {mse}")




ValueError: could not convert string to float: 'RM'